In [1]:
import requests
from bs4 import BeautifulSoup
import os
import polars as pl
from zipfile import ZipFile
from datetime import datetime

import source
from extractions import (
    download_files,
    read_files_from_landing,
    write_files_to_lake,
    read_files_from_lake
)


In [3]:
#cvmDownloadFiles('itr')

Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2011.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2012.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2013.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2014.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2015.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2016.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2017.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2018.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2019.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2020.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2021.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2022.zip
Downloaded file: /media/glauco/HDD1/followTheMarket/itr_cia_aberta_2023.zip


In [2]:
df = read_files_from_landing('itr','dre')

In [3]:
write_files_to_lake(df, 'raw', 'dre')

In [12]:
df = read_files_from_lake('raw', 'dre')
df.schema

OrderedDict([('CNPJ_CIA', Utf8),
             ('DT_REFER', Date),
             ('VERSAO', Int64),
             ('DENOM_CIA', Utf8),
             ('CD_CVM', Int64),
             ('GRUPO_DFP', Utf8),
             ('MOEDA', Utf8),
             ('ESCALA_MOEDA', Utf8),
             ('ORDEM_EXERC', Utf8),
             ('DT_INI_EXERC', Date),
             ('DT_FIM_EXERC', Date),
             ('CD_CONTA', Utf8),
             ('DS_CONTA', Utf8),
             ('VL_CONTA', Float64),
             ('ST_CONTA_FIXA', Utf8)])

In [94]:
dff = (
    df
    .with_columns(
        rn_quarter = (pl.col('DT_INI_EXERC'))
        .rank('ordinal',descending=True)
        .over(['CNPJ_CIA', 'DT_REFER', 'CD_CONTA'])
        )
    .filter(pl.col('rn_quarter')==1)
    .filter(pl.col('ORDEM_EXERC')=='ÚLTIMO')
    .filter(pl.col('DT_FIM_EXERC')==datetime(2022, 6, 30))
    .filter(pl.col('CD_CVM')==1023)
    .sort(pl.col('CD_CONTA'))
    .select(
        pl.col('DT_REFER'),
        pl.col('CD_CVM'),
        pl.col('CD_CONTA'),
        pl.col('DS_CONTA'),
        pl.col('VL_CONTA')
    )
)
dff

DT_REFER,CD_CVM,CD_CONTA,DS_CONTA,VL_CONTA
date,i64,str,str,f64
2022-06-30,1023,"""3.01""","""Receitas de In…",5.7469566e7
2022-06-30,1023,"""3.01.01""","""Receita de Jur…",5.7469566e7
2022-06-30,1023,"""3.02""","""Despesas de In…",-3.949672e7
2022-06-30,1023,"""3.02.01""","""Despesa de Jur…",-3.949672e7
2022-06-30,1023,"""3.03""","""Resultado Brut…",1.7972846e7
2022-06-30,1023,"""3.04""","""Outras Despesa…",-6.340451e6
2022-06-30,1023,"""3.04.01""","""Despesa de Pro…",-1.359403e6
2022-06-30,1023,"""3.04.02""","""Receitas de Pr…",5.951206e6
2022-06-30,1023,"""3.04.03""","""Despesas com P…",-6.343621e6


In [95]:
dff.write_csv(source.OUTPUTS_DIR)

Receita
> 3.01

Lucro
> 3.11